In [1]:
import pandas as pd
import numpy
import os
from datetime import datetime
from datetime import timedelta
%matplotlib notebook
import matplotlib.pyplot as plt
import pymultinest
import json
import corner
import warnings
from glob import glob




In [2]:
class simple_multinest():
    
    def __init__(self, x, y, model, parameters):
        
        self._x = x
        self._y = y
        self._model = model
        self._parameters = parameters
        
        #parameters is a dictionar with { 'par_name': {'min': val, 'max': val}}
    
    def create_uniform_prior(self, par_name):
        
        def uniform_transform(x):
            return self._parameters[par_name]['min']+ x * (self._parameters[par_name]['max']- self._parameters[par_name]['min'])
        
        self._parameters[par_name].update(transform = uniform_transform)
        self._parameters[par_name].update(aftertransform=lambda x: x)
        
    def make_all_uniform(self):
        
        for kk in self._parameters.keys():
            self.create_uniform_prior(kk)
    
    
    def test_statistics(self, par):
        
        #print(par)
        s = self._model(self._x, par)
    
        res = s - self._y + self._y * numpy.log( self._y / s)
    
        
        
        return 2 * numpy.sum(res)
    
    def log_likelihood(self, cube, ndim, nparams):
        
        par = {}
        for i, k in enumerate(self._parameters.keys()):
            t=self._parameters[k]
            par.update({k : t['aftertransform'](cube[i])})
        
        #print(par)
        return -0.5 * self.test_statistics(par)
    
    def create_prior_function(self):
        """
        Creates a single prior transformation function from parameters
        """

        def prior(cube, ndim, nparams):
                try:
                        for i, k in enumerate(self._parameters.keys()):
                                t=self._parameters[k]
                                transform = t['transform']
                                cube[i] = transform(cube[i])
                except Exception as e:
                        print('ERROR: Exception in prior function. Faulty transformations specified!')
                        print('ERROR:', e)
                        raise e

        return prior
    
    def nested_run(self, prior_function = None, sampling_efficiency = 0.3,
        n_live_points = 400, evidence_tolerance = 0.5,
        outputfiles_basename = 'nested-', verbose=True, clean=False, **kwargs):
        
        n_params = len(self._parameters)
        
        prior_function=self.create_prior_function()
        
        if clean:
            to_be_cleaned=glob(outputfiles_basename+'*')
            for ff in to_be_cleaned:
                os.remove(ff)
        
        pymultinest.run(self.log_likelihood, prior_function, n_params,
                sampling_efficiency = sampling_efficiency, n_live_points = n_live_points,
                outputfiles_basename = outputfiles_basename,
                verbose=verbose, **kwargs)
        
        paramnames = [str(k) for k in self._parameters.keys()]
        json.dump(paramnames, open('%s-params.json' % outputfiles_basename, 'w'), indent=4)
        
        
        self._analyzer = pymultinest.Analyzer(n_params = len(self._parameters),
                                                outputfiles_basename = outputfiles_basename)
        
        return outputfiles_basename

    def marginal_plots(self,minweight=1e-4, **kwargs):
        """
        Create marginal plots
        
        * analyzer: A instance of pymultinest.Analyzer
        * d: if more than 20 dimensions, by default only 1d marginal distributions
           are plotted. set d=2 if you want to force a 2d matrix plot
        
        """
        prefix = self._analyzer.outputfiles_basename
        parameters = json.load(open(prefix + '-params.json'))

        data = analyzer.get_data()[:,2:]
        weights = analyzer.get_data()[:,0]

        mask = weights > minweight
        
        
        with warnings.catch_warnings():
                warnings.simplefilter("ignore")
                corner.corner(data[mask,:], weights=weights[mask],
                        labels=parameters, show_titles=True, **kwargs)

        plt.savefig(prefix + 'corner.pdf')
        
    def model_plot(self, nsample=100, xlabel='Days', ylabel='Poissonian variable', yscale='log', title='Title' ):
        
        posterior = self._analyzer.get_equal_weighted_posterior()
        print(len(posterior))
        if nsample > len(posterior):
            raise RuntimeError('Cannot plot more than sample')
        
        chosen = numpy.random.choice(numpy.arange(len(posterior)), replace=False, size=nsample)
        posterior = posterior[chosen,:]
        
        
        plt.figure()
        plt.errorbar(self._x, self._y, yerr=numpy.sqrt(self._y), marker='o', linestyle='')
        plt.xlabel(xlabel)
        plt.ylabel(ylabel)
        plt.yscale(yscale)
        plt.title(title)
        
        xx=numpy.linspace(self._x.min(), self._x.max()+1, nsample)
        
        for k, row in enumerate(posterior):
            par = {}
            for i, k in enumerate(self._parameters.keys()):
                t=self._parameters[k]
                par.update({k : t['aftertransform'](row[i])})
            yy=self._model(xx, par)
            
            plt.plot(xx,yy, 'r-', alpha=0.5, linewidth=3)
        

In [3]:
#data = pd.read_csv("https://raw.githubusercontent.com/daenuprobst/covid19-cases-switzerland/master/covid19_cases_switzerland.csv")
data = pd.read_csv("https://raw.githubusercontent.com/daenuprobst/covid19-cases-switzerland/master/covid19_fatalities_switzerland.csv")
data

,Date,AG,AI,AR,BE,BL,BS,FR,GE,GL,...,SO,SZ,TG,TI,UR,VD,VS,ZG,ZH,CH
0,2020-03-06,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
1,2020-03-07,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
2,2020-03-08,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
3,2020-03-09,NaN,NaN,NaN,0.0,NaN,NaN,NaN,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
4,2020-03-10,NaN,NaN,NaN,0.0,NaN,NaN,NaN,2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
5,2020-03-11,NaN,NaN,NaN,0.0,NaN,NaN,NaN,2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
6,2020-03-12,NaN,NaN,NaN,0.0,NaN,NaN,NaN,2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
7,2020-03-13,NaN,NaN,NaN,0.0,NaN,NaN,NaN,2,NaN,...,NaN,NaN,NaN,NaN,NaN,2.0,1.0,NaN,0.0,NaN
8,2020-03-14,NaN,NaN,NaN,0.0,NaN,NaN,NaN,2,NaN,...,NaN,NaN,NaN,NaN,NaN,3.0,1.0,NaN,NaN,NaN
9,2020-03-15,NaN,NaN,NaN,0.0,NaN,2.0,NaN,4,NaN,...,NaN,NaN,NaN,6.0,NaN,4.0,1.0,NaN,NaN,NaN


In [4]:
canton='VD'

data = data.filter(items=['Date', canton])  # only keep data for one canton (or whole CH) in dataframe
data.rename(columns={canton: "TotalPosTests1"}, inplace=True)
data = data.dropna()  # remove dates without available data (na)
newdata = data
startrow = 0
for x in data['TotalPosTests1']:
    if x == 0:
        startrow+=1
newdata = newdata.iloc[startrow:]
inhabitantsCH = 8570000.0
datearray = newdata['Date'].to_numpy()
for x in datearray:
    x = datetime.strptime(x, '%Y-%m-%d')
datetimearray = [datetime.strptime(x, '%Y-%m-%d').date() for x in datearray]
try:
    firstdate = datetimearray[1]
except IndexError:
    sys.exit('No cases in database for {}.'.format(canton))

datesincezero = [float((x - firstdate).days) + 1.0 for x in datetimearray]
poscasearray = newdata['TotalPosTests1'].to_numpy().tolist()
x = numpy.array(datesincezero)
y = numpy.array(poscasearray)

In [14]:
# parameters = {
#     'break' : {'min': 1., 'max' : 9.} , 
#     't1' : {'min': -40, 'max' : 0.} , 
#     't2' : {'min': -200, 'max' : 0.} ,  
#     'b1'  : {'min': 0, 'max' : 2.} , 
#     'b2' : {'min': 0, 'max' : 2.} 
# }

parameters = {
    't1' : {'min': -40, 'max' : 0.} , 
    'b1'  : {'min': 0, 'max' : 2.} 
}



def pl(x,par):
    t1  = par['t1']
    b1  = par['b1']
    
    return numpy.exp(b1*(x-t1))
    


def broken_pl(x, par):
    
    #print(par)
    
    break_value = par['break']
    t1  = par['t1']
    t2  = par['t2']
    b1  = par['b1']
    b2  = par['b2']
    
    ind = x > break_value
    
    
    
    y = numpy.exp(b1*(x-t1))
    
    if numpy.sum(ind) >0:
        y[ind]=numpy.exp(b2*(x[ind]-t2))
    
    return y

outputfiles_basename = 'nested-dead-'+canton+'-'
# my_multinest=simple_multinest(x, y, broken_pl, parameters)
my_multinest=simple_multinest(x, y, pl, parameters)

my_multinest.make_all_uniform()

my_multinest.nested_run(outputfiles_basename = outputfiles_basename, clean=True)

    
    
    

  analysing data from nested-dead-VD-.txt


'nested-dead-VD-'

In [15]:

# analyse results
print('analysing results...')
analyzer = pymultinest.Analyzer(n_params = len(parameters),
        outputfiles_basename = outputfiles_basename)
s = analyzer.get_stats()
# store information in readable, hierarchical format
json.dump(s, open(outputfiles_basename + 'stats.json', 'w'), indent=4)

#   print out summary
print()
print()
print('Parameter estimation summary')
print('****************************')
print()
print(' %20s: median, 10%%, q90%% quantile' % ('parameter name'))
print(' ', '-'*20)
for t, m in zip(parameters.keys(), s['marginals']):
        print(' %20s: %.3f  %.3f %.3f ' % (t, m['median'], m['q10%'], m['q90%']))
print()
print()
print('Model evidence: ln(Z) = %.2f +- %.2f' % (s['global evidence'], s['global evidence error']))
print()


analysing results...
  analysing data from nested-dead-VD-.txt


Parameter estimation summary
****************************

       parameter name: median, 10%, q90% quantile
  --------------------
                   t1: -4.607  -8.160 -2.297 
                   b1: 0.197  0.151 0.244 


Model evidence: ln(Z) = -7.56 +- 0.02



1369


<IPython.core.display.Javascript object>


In [16]:
xlabel= "Days since %s"%firstdate.strftime('%Y-%m-%d')
ylabel= '# of positive tests (BAG)'
yscale='log'
my_multinest.model_plot(100, xlabel, ylabel, yscale, title=canton)



<IPython.core.display.Javascript object>


In [17]:
my_multinest.marginal_plots()